# Exemplo amostragem

## Carregar módulos

In [1]:
import numpy as np
import pandas as pd
import random as rd
from sklearn.model_selection import train_test_split

## Carregar dados

In [2]:
dados = pd.read_csv('dados_renda_municipios.csv',
                    sep = ',',
                    decimal = '.',
                    encoding = 'utf-8')

In [3]:
dados.head()

,UF,Municipio,RDPC
0,Rondônia,ALTA FLORESTA D'OESTE,476.99
1,Rondônia,ARIQUEMES,689.95
2,Rondônia,CABIXI,457.17
3,Rondônia,CACOAL,738.06
4,Rondônia,CEREJEIRAS,577.18


## Amostragem

### Resultados piloto

Neste exemplo iremos utilizar os dados do censo como parâmetros de amostra piloto.

Deste modo, reduziriamos os custos de obter uma amostra piloto.

- Escolher UF

In [4]:
uf = 'São Paulo'

In [5]:
dados_municipio = dados[dados['UF'] == uf].reset_index()

In [6]:
dados_municipio.shape

(645, 4)

- Classificação de renda

In [7]:
dados_municipio['classe_renda'] = pd.qcut(dados_municipio['RDPC'], 4, labels = ['D', 'C', 'B', 'A'])

In [8]:
dados_municipio.head()

,index,UF,Municipio,RDPC,classe_renda
0,3266,São Paulo,ADAMANTINA,975.43,A
1,3267,São Paulo,ADOLFO,661.65,C
2,3268,São Paulo,AGUAÍ,636.07,C
3,3269,São Paulo,ÁGUAS DA PRATA,853.39,A
4,3270,São Paulo,ÁGUAS DE LINDÓIA,730.13,B


- Métricas da renda da amostra piloto (Censo)

In [9]:
dados_piloto = dados_municipio.agg(media_RDPC = pd.NamedAgg('RDPC', 'mean'),
                                   dp_RDPC = pd.NamedAgg('RDPC', 'std'),
                                   N = pd.NamedAgg('RDPC', 'count'))

In [10]:
dados_piloto

,RDPC
media_RDPC,713.926155
dp_RDPC,197.398270
N,645.000000


In [11]:
dados_piloto_classe = dados_municipio.groupby('classe_renda') \
                                     .agg(media_RDPC = pd.NamedAgg('RDPC', 'mean'),
                                          dp_RDPC = pd.NamedAgg('RDPC', 'std'),
                                          N = pd.NamedAgg('RDPC', 'count')) \
                                     .reset_index()

In [12]:
dados_piloto_classe['percent'] = dados_piloto_classe['N'] / sum(dados_piloto_classe['N'])

In [13]:
dados_piloto_classe

,classe_renda,media_RDPC,dp_RDPC,N,percent
0,D,517.531296,58.889964,162,0.251163
1,C,637.659259,28.839032,162,0.251163
2,B,736.007313,34.142763,160,0.248062
3,A,966.337453,204.484141,161,0.249612


### Obter amostra

- Fórmula amostra

In [14]:
def formula_amostra_discreta(N, Z, ME):
  n = (Z**2 * 0.25 * N) / ((ME**2 * (N-1)) + (Z**2 * 0.25))
  return int(n)

In [15]:
def formula_amostra_continua(N, S, Z, ME):
  n = (Z**2 * S**2 * N) / ((ME**2 * (N-1)) + (Z**2 * S**2))
  return int(n)

No caso iremos utilizar a fórmula da amostra contínua

In [16]:
N = 645
Z = 1.96
S = 197.40
ME = 25

In [17]:
n = formula_amostra_continua(N, S, Z, ME)
n

174

### Métodos Amostragem

- Amostra aleatória simples

In [18]:
dados_amostra_simples = dados_municipio.sample(n=n, random_state=1245)

In [19]:
dados_amostra_simples.head()

,index,UF,Municipio,RDPC,classe_renda
276,3542,São Paulo,JABOTICABAL,1015.51,A
494,3760,São Paulo,RIO GRANDE DA SERRA,563.57,D
12,3278,São Paulo,ALTO ALEGRE,560.30,D
581,3847,São Paulo,SOROCABA,1107.19,A
207,3473,São Paulo,GUARAREMA,748.79,B


In [20]:
dados_amostra_simples.shape

(174, 5)

In [21]:
dados_amostra_simples.agg(media_RDPC = pd.NamedAgg('RDPC', 'mean'),
                          dp_RDPC = pd.NamedAgg('RDPC', 'std'),
                          N = pd.NamedAgg('RDPC', 'count'))

,RDPC
media_RDPC,714.228851
dp_RDPC,197.027325
N,174.000000


In [22]:
dados_piloto

,RDPC
media_RDPC,713.926155
dp_RDPC,197.398270
N,645.000000


In [23]:
resumo = dados_amostra_simples.groupby('classe_renda') \
                              .agg(media_RDPC = pd.NamedAgg('RDPC', 'mean'),
                                    dp_RDPC = pd.NamedAgg('RDPC', 'std'),
                                    N = pd.NamedAgg('RDPC', 'count')) \
                              .reset_index()

In [24]:
resumo['percent'] = resumo['N'] / sum(resumo['N'])

In [25]:
resumo

,classe_renda,media_RDPC,dp_RDPC,N,percent
0,D,501.869744,66.576554,39,0.224138
1,C,639.571136,28.383709,44,0.252874
2,B,730.668800,33.343062,50,0.287356
3,A,976.300732,201.544108,41,0.235632


In [26]:
dados_piloto_classe

,classe_renda,media_RDPC,dp_RDPC,N,percent
0,D,517.531296,58.889964,162,0.251163
1,C,637.659259,28.839032,162,0.251163
2,B,736.007313,34.142763,160,0.248062
3,A,966.337453,204.484141,161,0.249612


- Amostra estratificada

In [27]:
dados_amostra_estrat = train_test_split(dados_municipio,
                                        test_size = n,
                                        random_state=1245,
                                        stratify=dados_municipio['classe_renda'])[1]

In [28]:
dados_amostra_estrat.head()

,index,UF,Municipio,RDPC,classe_renda
251,3517,São Paulo,ITAJU,609.08,C
468,3734,São Paulo,QUATÁ,648.64,C
363,3629,São Paulo,NAZARÉ PAULISTA,554.18,D
339,3605,São Paulo,MIRANTE DO PARANAPANEMA,516.30,D
238,3504,São Paulo,INÚBIA PAULISTA,576.91,D


In [29]:
dados_amostra_estrat.shape

(174, 5)

In [30]:
dados_amostra_estrat.agg(media_RDPC = pd.NamedAgg('RDPC', 'mean'),
                          dp_RDPC = pd.NamedAgg('RDPC', 'std'),
                          N = pd.NamedAgg('RDPC', 'count'))

,RDPC
media_RDPC,718.882874
dp_RDPC,205.568258
N,174.000000


In [31]:
dados_piloto

,RDPC
media_RDPC,713.926155
dp_RDPC,197.398270
N,645.000000


In [32]:
resumo_estrat = dados_amostra_estrat.groupby('classe_renda') \
                                    .agg(media_RDPC = pd.NamedAgg('RDPC', 'mean'),
                                          dp_RDPC = pd.NamedAgg('RDPC', 'std'),
                                          N = pd.NamedAgg('RDPC', 'count')) \
                                    .reset_index()

In [33]:
resumo_estrat['percent'] = resumo_estrat['N'] / sum(resumo_estrat['N'])

In [34]:
resumo_estrat

,classe_renda,media_RDPC,dp_RDPC,N,percent
0,D,530.192500,42.159454,44,0.252874
1,C,635.421591,28.204496,44,0.252874
2,B,733.693953,34.054255,43,0.247126
3,A,982.552558,232.962593,43,0.247126


In [35]:
dados_piloto_classe

,classe_renda,media_RDPC,dp_RDPC,N,percent
0,D,517.531296,58.889964,162,0.251163
1,C,637.659259,28.839032,162,0.251163
2,B,736.007313,34.142763,160,0.248062
3,A,966.337453,204.484141,161,0.249612


In [36]:
982-966

16